In [26]:
import os, sys
import numpy as np
import torch

In [27]:
d = os.getcwd()
p = os.path.dirname(d)

sys.path.append(p)

In [28]:
from src.dynamics import RobotWorld
from src.lqr import Lqr
from src.algorithms.dpgpd import Dpgpd
from src.algorithms.adpgpd import ADpgpd
from src.sampling import Sampler

# 1 - Parameters

In [54]:
ds = 4
da = 2

eta = 0.01
tau = 0.01
gamma = 0.9
alpha = 1.0

epochs = 2_000
n_pe = 100
n_rho = 1_000_000

b = - 90

G1 = - torch.tensor([
    [1.0, 0, 0, 0],
    [0, 1.0, 0, 0],
    [0, 0, .1, 0],
    [0, 0, 0, .1]
])

R1 =  - torch.tensor([
    [0.1, 0],
    [0, 0.1],
]).double()

H = - (tau / 2) * torch.eye(da)

G2 = - torch.tensor([
    [.1, 0, 0, 0],
    [0, .1, 0, 0],
    [0, 0, 1.0, 0],
    [0, 0, 0, 1.0]
])

R2 = - torch.tensor([
    [0.1, 0],
    [0, 0.1],
]).double()

env = RobotWorld(range_pos=[-10., 10.], range_vel=[-.1, .1])
lqr = Lqr(env.A, env.B, gamma)
sampler = Sampler(env, gamma)

# 2 - Solution

In [55]:
P = lqr.calculate_optimal_P(G1, R1)
K = lqr.calculate_optimal_K(R1, P)
opt_lqr = sampler.estimate_V_rho_closed(P, 1_000_000)
print(f"Optimal solution: {opt_lqr}")

Optimal solution: -569.835687761704


In [56]:
dpgpd = Dpgpd(env, eta, tau, gamma, b, G1, G2, R1, R2, H)
_, _, pl_exact, dl_exact = dpgpd.train_constrained(epochs, n_rho)

In [61]:
adpgpd = ADpgpd(env, eta, tau, gamma, b, alpha, G1, G2, R1, R2, H)
_, _, pl_approx, dl_approx = adpgpd.train_constrained(epochs, n_pe, n_rho)

In [62]:
np.save('../results/vel_primal_exact.npy', pl_exact)
np.save('../results/vel_dual_exact.npy', dl_exact)
np.save('../results/vel_primal_approx.npy', pl_approx)
np.save('../results/vel_dual_approx.npy', dl_approx)

# 3 - Varying eta

In [72]:
pl_eta_exact, dl_eta_exact = [], []
etas = [0.05, 0.01, 0.001]
for eta in etas:
    dpgpd = Dpgpd(env, eta, tau, gamma, b, G1, G2, R1, R2, H)
    _, _, pl, dl = dpgpd.train_constrained(epochs, n_rho)
    pl_eta_exact.append(pl), dl_eta_exact.append(dl)
pl_eta_exact, dl_eta_exact = np.array(pl_eta_exact), np.array(dl_eta_exact)

In [73]:
pl_eta_approx, dl_eta_approx = [], []
etas = [0.05, 0.01, 0.001]
for eta in etas:
    adpgpd = ADpgpd(env, eta, tau, gamma, b, alpha, G1, G2, R1, R2, H)
    _, _, pl, dl = adpgpd.train_constrained(epochs, n_pe, n_rho)
    pl_eta_approx.append(pl), dl_eta_approx.append(dl)
pl_eta_approx, dl_eta_approx = np.array(pl_eta_approx), np.array(dl_eta_approx)

In [74]:
np.save('../results/vel_primal_exact_eta.npy', pl_eta_exact)
np.save('../results/vel_dual_exact_eta.npy', dl_eta_exact)
np.save('../results/vel_primal_approx_eta.npy', pl_eta_approx)
np.save('../results/vel_dual_approx_eta.npy', dl_eta_approx)

# 4 - Varying tau

In [75]:
pl_tau_exact, dl_tau_exact = [], []
taus = [1.0, 0.5, 0.1, 0.01]
eta = 0.01
epochs = 2_000
for tau in taus:
    H = - (tau / 2) * torch.eye(da)
    dpgpd = Dpgpd(env, eta, tau, gamma, b, G1, G2, R1, R2, H)
    _, _, pl, dl = dpgpd.train_constrained(epochs, n_rho)
    pl_tau_exact.append(pl), dl_tau_exact.append(dl)
pl_tau_exact, dl_tau_exact = np.array(pl_tau_exact), np.array(dl_tau_exact)

In [76]:
pl_tau_approx, dl_tau_approx = [], []
taus = [1.0, 0.5, 0.1, 0.01]
eta = 0.01
epochs = 2_000
for tau in taus:
    H = - (tau / 2) * torch.eye(da)
    adpgpd = ADpgpd(env, eta, tau, gamma, b, alpha, G1, G2, R1, R2, H)
    _, _, pl, dl = adpgpd.train_constrained(epochs, n_pe, n_rho)
    pl_tau_approx.append(pl), dl_tau_approx.append(dl)
pl_tau_approx, dl_tau_approx = np.array(pl_tau_approx), np.array(dl_tau_approx)

In [77]:
np.save('../results/vel_primal_exact_tau.npy', pl_tau_exact)
np.save('../results/vel_dual_exact_tau.npy', dl_tau_exact)
np.save('../results/vel_primal_approx_tau.npy', pl_tau_approx)
np.save('../results/vel_dual_approx_tau.npy', dl_tau_approx)

# 5 - Optimality gap

In [6]:
etas = [0.0005, 0.0002, 0.0001]
epochs = 50_000
n_rho = 1_000_000

V_primal_opt = - 596

dg_eta_exact = []
for eta in etas:
    dpgpd = Dpgpd(env, eta, tau, gamma, b, G1, G2, R1, R2, H)
    dg = dpgpd.evaluate_duality_gap(V_primal_opt, epochs, n_rho)
    dg_eta_exact.append(dg)
dg_eta_exact = np.array(dg_eta_exact)

In [9]:
etas = [0.0005, 0.0002, 0.0001]
epochs = 50_000
n_rho = 1_000_000

V_primal_opt = - 596.0

dg_eta_inexact = []
for eta in etas:
    adpgpd = ADpgpd(env, eta, tau, gamma, b, alpha, G1, G2, R1, R2, H)
    dg = adpgpd.evaluate_duality_gap(V_primal_opt, epochs, n_pe, n_rho)
    dg_eta_inexact.append(dg)
dg_eta_inexact = np.array(dg_eta_inexact)

In [11]:
np.save('../results/vel_primal_exact_dg.npy', dg_eta_exact)
np.save('../results/vel_primal_inexact_dg.npy', dg_eta_inexact)